In [20]:
import pandas as pd
from datasets import load_dataset, get_dataset_config_names
from huggingface_hub import HfApi
import numpy as np
import pdb
import logging

In [21]:
api = HfApi()

In [22]:
data = api.list_datasets(filter='task_categories:translation') # consider saving paper info too; if available! can modify later
mt_datasets = [d for d in data] 
mt_df = pd.read_csv('data/mt_hf.csv')

In [23]:
# mt_datasets

In [1]:
from datasets import load_dataset

ds = load_dataset("thevox/en-nb-10k")

DatasetNotFoundError: Dataset 'thevox/en-nb-10k' is a gated dataset on the Hub. You must be authenticated to access it.

In [25]:
data = api.list_datasets(search='JA_audio_JA_text_180k_samples') # consider saving paper info too; if available! can modify later

In [26]:
data = [d for d in data]

In [27]:
if 'language:ja' not in data[0].tags: print(True)

In [28]:
# data[0].tags

In [29]:
two = mt_df[mt_df['# Languages'] == 2]
one = mt_df[mt_df['# Languages'] == 1]
zero = mt_df[mt_df['# Languages'] == 0]
more = mt_df[mt_df['# Languages'] > 2]
print(f"Two langs: {len(two)}\nOne lang: {len(one)}\nZero langs: {len(zero)}\nMore: {len(more)}")

Two langs: 642
One lang: 234
Zero langs: 95
More: 478


In [30]:
test = eval(two.iloc[0]['Supported Languages'])

In [31]:
"-".join(test)

'en-ja'

In [32]:
# mt_df

In [33]:
COLS2 = ['Author/Dataset', 'Language Pair', '# Train Set', '# Development Set', '# Test Set']

logging.basicConfig(level=logging.INFO, format='%(asctime)s | %(message)s')

def log_missing_data(dataset_name):
    with open("missing_datasets.txt", "a") as f:
        f.write(f"{dataset_name}\n")

def get_pairs(mt_data):
    data = mt_data[mt_data['# Languages'].isin((1,2))]
    mt_data = data[['Author/Dataset', 'Dataset Type', 'Supported Languages']]
 
    data = []
    for _, row in mt_data.iterrows():
        if pd.isna(row['Dataset Type']):
            continue
        else:
            try:
                ID = row['Author/Dataset']
                langs = eval(row['Supported Languages'])

                if len(langs) > 1:
                    pair = langs[0] + '-' + langs[1]
                else:
                    pair = langs[0]

                datum = [ID, pair, 0, 0, 0]
                logging.info(f"Loading dataset: {ID}")
                ds = load_dataset(ID)
            
                for split in ds:
                    if split == 'train':
                        datum[2] = ds[split].num_rows
                    elif split == 'validation':
                        datum[3] = ds[split].num_rows
                    else:
                        datum[4] = ds[split].num_rows

                data.append(datum)
            except ValueError:
                logging.info(f"Error loading dataset: {ID}") 
#                 pdb.set_trace()
            except NameError:
                logging.info(f"TEST loading dataset: {ID}") 
            except:
                logging.info(f"{ID} is a gated dataset on the hub. You must be authenticated to access it.")
                log_missing_data(row['Author/Dataset']) 


    df = pd.DataFrame(data, columns=COLS2)
    return df

In [34]:
# load_dataset('1TuanPham/T-VisEx26-dataset-uncensored')

In [35]:
# load_dataset('IWSLT/mt_eng_vietnamese') ValueError
# load_dataset('airesearch/scb_mt_enth_2020')
# load_dataset('RobotsMaliAI/bayelemabaga')
# load_dataset('Helsinki-NLP/ted_iwlst2013')

In [36]:
# get_pairs(mt_df)

In [185]:
import pandas as pd

# Sample old data
old_data = pd.DataFrame({
    'Author/Dataset': ['A', 'B', 'C'],
    'Dataset Type': ['Type1', 'Type2', 'Type3']
})

# Sample new data
new_data = pd.DataFrame({
    'Author/Dataset': ['A', 'C', 'D']
})

# # Perform the mapping
# new_data['Dataset Type'] = new_data['Author/Dataset'].map(old_data.set_index('Author/Dataset')['Dataset Type'])

# # Print the new data with the added 'Dataset Type' column
# print(new_data)


In [196]:
test = old_data.set_index('Author/Dataset')['Dataset Type']
test

Author/Dataset
A    Type1
B    Type2
C    Type3
Name: Dataset Type, dtype: object

In [198]:
test2 = new_data['Author/Dataset'].map(test)
test2

0    Type1
1    Type3
2      NaN
Name: Author/Dataset, dtype: object

In [199]:
new_data['Dataset Type'] = test2
new_data

,Author/Dataset,Dataset Type
0,A,Type1
1,C,Type3
2,D,NaN


In [184]:
import re
import csv

# Input and output files
input_file = 'train.csv'
output_file = 'corrected.csv'

# Regular expression to match "Translation in [Language]: <text>"
pattern = r'Translation in ([a-zA-Z]+):\s*([^Translation]*)'

with open(input_file, 'r') as infile, open(output_file, 'w', newline='') as outfile:
    reader = infile.readlines()
    writer = csv.writer(outfile)

    # Initialize a header for the CSV
    header_written = False

    for line in reader:
        matches = re.findall(pattern, line)
        if matches:
            # Extract languages and translations
            languages = [lang for lang, _ in matches]
            translations = [text.strip() for _, text in matches]

            # Write the header dynamically if not already written
            if not header_written:
                writer.writerow(languages)
                header_written = True

            # Write the translations row
            writer.writerow(translations)


In [180]:
df = pd.read_csv('train.csv')

/var/folders/xx/l7vsjhy55856z9s9d1l4__580000gn/T/ipykernel_49020/2436019669.py:1: DtypeWarning: Columns (13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('train.csv')


In [183]:
df.iloc[159830]

prompt\tresult                    Translation in English: “Our Lord
Unnamed: 1               we have believed in what You have revealed
Unnamed: 2                       and we have followed the Messenger
Unnamed: 3         so count us among the witnesses.” Translation...
Unnamed: 4         (உன்னுடைய) இத்தூதரை நாங்கள் பின்பற்றுகிறோம்;...
                                        ...                        
Unnamed: 69                                                     NaN
Unnamed: 70                                                     NaN
Unnamed: 71                                                     NaN
Unnamed: 72                                                     NaN
Unnamed: 73                                                     NaN
Name: 159830, Length: 74, dtype: object

In [177]:
with open('train.csv') as f:
    lines = f.readlines()

In [133]:
pre = [line.rstrip(',\n') for line in lines]

In [178]:
lines[90431]

'Translation in English: Error while retrieving a key used for encryption. You may solve such a problem with one of the following methods: in a terminal either set the proper DVD region code for your CD/DVD player with the """"regionset %s"""" command or run the """"DVDCSS_METHOD=title brasero --no-existing-session"""" command ",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,\n'

In [174]:
pre[90431]

'Translation in English: Error while retrieving a key used for encryption. You may solve such a problem with one of the following methods: in a terminal either set the proper DVD region code for your CD/DVD player with the """"regionset %s"""" command or run the """"DVDCSS_METHOD=title brasero --no-existing-session"""" command "'

In [172]:
data = []
for idx, text in enumerate(pre[1:]):
    try:
        parts = text.split('Translation in')
        lang1 = "Translation in" + parts[1].rstrip()
        lang2 = "Translation in" + parts[2]
        data.append([lang1, lang2])
    except:
        pdb.set_trace()

> /var/folders/xx/l7vsjhy55856z9s9d1l4__580000gn/T/ipykernel_49020/3800042767.py(2)<module>()
      1 data = []
----> 2 for idx, text in enumerate(pre[1:]):
      3     try:
      4         parts = text.split('Translation in')
      5         lang1 = "Translation in" + parts[1].rstrip()

ipdb> idx
90430
ipdb> q


In [167]:
pre[1]

'"Translation in Afrikaans: Kom Translation in English:\tCome on."'

In [155]:
test = pre[1].split('Translation in')
test

['"', ' Afrikaans: Kom ', ' English:\tCome on."']

In [160]:
test2 = test[1].rstrip()
test2

' Afrikaans: Kom'

In [162]:
"Translation in" + test2

'Translation in Afrikaans: Kom'

In [157]:
"Translation in".join(test[1])

' Translation inATranslation infTranslation inrTranslation iniTranslation inkTranslation inaTranslation inaTranslation innTranslation insTranslation in:Translation in Translation inKTranslation inoTranslation inmTranslation in '

In [150]:
test = re.findall(r'Translation in \w+:', pre[1])

In [151]:
test

['Translation in Afrikaans:', 'Translation in English:']

In [126]:
test = re.findall(r'Translation in \w+: ', lines[1])
test

['Translation in Afrikaans:', 'Translation in English:']

In [127]:
test1 = lines[1].strip('",\n')
test1

'Translation in Afrikaans: Kom Translation in English:\tCome on.'

In [128]:
re.sub(test[0], "", lines[1])

'" Kom Translation in English:\tCome on.",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,\n'

In [122]:
test

['Translation in Afrikaans: ']

In [ ]:
re.findall(lines[1], '')

In [77]:
test = lines[1]
objects = test[1:].split('Translation in English:')

In [67]:
test = objects[1].lstrip('\t')

In [75]:
test.rstrip('\n').rstrip('",')

'Come on.'

In [83]:
objects[1].lstrip('\t').rstrip('",\n"')

'Come on.'

In [87]:
data = []
for idx, line in enumerate(lines[1:]):
    try:
        objects = line.split('Translation in English:')
        afrikanns = objects[0].strip()
    #     pdb.set_trace()
        english = objects[1].lstrip('\t').rstrip('",\n"')
        data.append([afrikanns, "Translation in English: " + english])
    except:
        pdb.set_trace()
        pass
    
    

> /var/folders/xx/l7vsjhy55856z9s9d1l4__580000gn/T/ipykernel_49020/573698212.py(11)<module>()
      9     except:
     10         pdb.set_trace()
---> 11         pass
     12 
     13 

ipdb> idx
90431
ipdb> line
'"ranslation in Kannada:""\t""ಗೂಢಲಿಪೀಕರಣಕ್ಕಾಗಿ ಬಳಸಲಾದ ಕೀಲಿಯನ್ನು ಪಡೆದುಕೊಳ್ಳುವಲ್ಲಿ ದೋಷ ಉಂಟಾಗಿದೆ. ಈ ಕೆಳಗಿನ ವಿಧಾನದ ಮೂಲಕ ಅಂತಹ ತೊಂದರೆಯನ್ನು ನೀವು ಸರಿಪಡಿಸಲು ಸಾಧ್ಯವಿರುತ್ತದೆ: ಟರ್ಮಿನಲ್\u200cನಲ್ಲಿ """"regionset %s"""" ಆಜ್ಞೆಯನ್ನು ಬಳಸಿಕೊಂಡು ಸೂಕ್ತವಾದ DVD ಪ್ರದೇಶ ಸಂಕೇತವನ್ನು ನಿಮ್ಮ CD/DVD ಚಾಲಕಕ್ಕೆ ಹೊಂದಿಸಿ ಅಥವ """"DVDCSS_METHOD=title brasero --no-existing-session"""" ಆಜ್ಞೆಯನ್ನು ಚಲಾಯಿಸಿ""",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,\n'
ipdb> q


In [40]:
# pd.read_csv('train.csv')

In [136]:
ds = load_dataset('Jour/Translation')

Failed to read file '/Users/weezygeezer/.cache/huggingface/hub/datasets--Jour--Translation/snapshots/b60f7964a8e708f9215d5c0f9a409397301cba20/train.csv' with error <class 'pandas.errors.ParserError'>: Error tokenizing data. C error: Expected 1 fields in line 19, saw 2

2025-01-09 22:10:39,431 | Failed to read file '/Users/weezygeezer/.cache/huggingface/hub/datasets--Jour--Translation/snapshots/b60f7964a8e708f9215d5c0f9a409397301cba20/train.csv' with error <class 'pandas.errors.ParserError'>: Error tokenizing data. C error: Expected 1 fields in line 19, saw 2



DatasetGenerationError: An error occurred while generating the dataset

In [ ]:
ds

In [95]:
ds = load_dataset('xri/BatakTobaNMT')

Generating train split: 0 examples [00:00, ? examples/s]

Failed to read file '/Users/weezygeezer/.cache/huggingface/hub/datasets--xri--BatakTobaNMT/snapshots/15810df98410d3ce47aa1a56f47c2e2e4bd6653a/bbc.tsv' with error <class 'pandas.errors.ParserError'>: Error tokenizing data. C error: Expected 4 fields in line 3969, saw 6

2025-01-09 00:32:55,388 | Failed to read file '/Users/weezygeezer/.cache/huggingface/hub/datasets--xri--BatakTobaNMT/snapshots/15810df98410d3ce47aa1a56f47c2e2e4bd6653a/bbc.tsv' with error <class 'pandas.errors.ParserError'>: Error tokenizing data. C error: Expected 4 fields in line 3969, saw 6



DatasetGenerationError: An error occurred while generating the dataset

In [107]:
with open('bbc.tsv', 'r') as file:
    lines = file.readlines()

with open('fixed_bbc.tsv', 'w') as file:
    for line in lines:
        # Split the line by tabs, count the number of fields, and fix if needed
        fields = line.split('\t')
        if len(fields) != 4:
            print(f"Fixing line: {line}")  # Debugging
            # You can either fix or skip lines based on your needs
            continue  # Skip line or fix it here
        file.write(line)


SyntaxError: invalid syntax (4199625348.py, line 1)

In [117]:
pd.read_csv('old_bbc.tsv', delimiter='\t')

ParserError: Error tokenizing data. C error: Expected 4 fields in line 3969, saw 6


In [119]:
pd.read_csv('bbc.tsv', delimiter='\t')

,bbc_4349,"In the midst of the farm, a trough, a symbol of hostility, stood out.","Di tengah-tengah pertanian, sebuah palung, simbol permusuhan, menonjol.","Di tonga-tonga parumaon, sada palakka, tanda hamusuon, juljul"
0,bbc_2764,"A story about him has been circulated, and his...","Sebuah cerita tentang dia telah beredar, dan a...","Sada sarita tarsingot ibana nunga beredar,jala..."
1,bbc_3640,"Tomorrow, I am certain, will be spent outside ...","Besok, aku yakin, akan dihabiskan di luar, di ...","marsogot, au pos, naeng dipahabis di ruar, di ..."
2,bbc_7842,"This painting, with its vibrant colors and int...","Lukisan ini, dengan warna-warna cerah dan deta...","Gorga on, dohot uarna-uarna tampak dohot detil..."
3,bbc_7439,"I had to lead him by the hand, gently nudging ...","Saya harus menuntun tangannya, dengan lembut m...","Ahu ikon manogu tangan na,dohot dalmet mangara..."
4,bbc_5651,"Tomorrow, I will use a fork to dig a hole for ...",Besok saya akan menggunakan garu untuk menggal...,Sogot naeng hupangke garu on laho mangongkal h...
...,...,...,...,...
7992,bbc_2656,"As the news of his condition spread, he had to...","Ketika berita tentang kondisinya menyebar, dia...","Pas barita taringot namasa i mangararat,ibana ..."
7993,bbc_7571,The customary sacrifice of a young lamb was on...,Salah satu ritualnya adalah pengorbanan anak d...,Sala sada ritualna ima mangurbanton anak ni bi...
7994,bbc_2579,"The teacher, the teacher of law, would say, 'T...","Guru itu, seorang guru ilmu hukum, akan berkat...","Guru i,sada guru parilmu uhum,naeng mandokkon,..."
7995,bbc_2713,There will be a time when I will stop weeping ...,Akan ada waktu ketika saya akan berhenti menan...,Ikkon adong tingki pas au naeng so tangis jala...


In [54]:
from datasets import DatasetNotFoundError

ImportError: cannot import name 'DatasetNotFoundError' from 'datasets' (/Users/weezygeezer/anaconda3/lib/python3.11/site-packages/datasets/__init__.py)

In [11]:
# df = pd.read_excel('language_pairs.xlsx')

In [13]:
# df.to_csv('language_pairs_external.csv', header=True, index=False)

In [14]:
# pip install datasets

In [238]:
ds = load_dataset('mesolitica/chatgpt-noisy-translation-b.cari.com.my')

README.md:   0%|          | 0.00/210 [00:00<?, ?B/s]

data.jsonl:   0%|          | 0.00/393M [00:00<?, ?B/s]

processed-b.cari.com.my.jsonl:   0%|          | 0.00/363M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Failed to load JSON from file '/Users/weezygeezer/.cache/huggingface/hub/datasets--mesolitica--chatgpt-noisy-translation-b.cari.com.my/snapshots/062da5399f3caf8559296579e49f429f9a6a7250/data.jsonl' with error <class 'pyarrow.lib.ArrowInvalid'>: JSON parse error: Column() changed from object to array in row 0
2025-01-10 22:21:14,819 | Failed to load JSON from file '/Users/weezygeezer/.cache/huggingface/hub/datasets--mesolitica--chatgpt-noisy-translation-b.cari.com.my/snapshots/062da5399f3caf8559296579e49f429f9a6a7250/data.jsonl' with error <class 'pyarrow.lib.ArrowInvalid'>: JSON parse error: Column() changed from object to array in row 0


DatasetGenerationError: An error occurred while generating the dataset

In [236]:
ds['train']

Dataset({
    features: ['user_id', 'path', 'text', 'durationMsec', 'sampleRate', 'speaker_gender', 'mother_tongue', 'date'],
    num_rows: 994
})